# Proof of Work
P2Pネットワークのような分散システムにおいて正しい情報を決定するために考案されたのが`Proof of Work(PoW)`。  
## Proof of Work のプロセス
取引データを受信して記録しておく  
↓  
取引データをまとめてブロックを生成  
↓  
ナンスを少しずつ変えながらハッシュ計算  
↓  
ブロックを完成させて他のノードに報告  
↓  
他のノードがブロックの中身とナンス値が正しいかを検証する  
↓  
ブロック生成に戻る。  
(マイニング報酬が支払われる)  

暗号学的ハッシュ関数の性質を巧みに使うことで、ブロックや格納されているデータの整合性を保っている。
## Proof of Work で改ざんが難しい理由
ブロックチェーンは改ざんに強いという性質を持っており、その根拠の１つが Proof of Work 。  
新しいブロックが生成されると、そのブロックはチェーンの先端に接続される。
その際に大量のハッシュ計算が行われており、例えば、ブロックチェーンのどこかの部分のデータを変更すると、そこから繋がるブロックの全てのデータが変わってしまう。  
もしも、変更したデータを正しいものにしたい場合は、連なるデータのハッシュ計算を再度し直して全て書き換える必要が出てくる。  
しかし、それにはProof of Work に基づいて極めて多くの計算リソースを投入しなければなりませんし、そのためには高性能のマシンが必要で多額の投資が必要になります。  
この時、そのような再計算が可能なリソースを確保できるのであれば、正規のマイニングを行なってマイニング報酬を得る方が経済的に合理的であると考えられることができます。
また、データを書き換えるような不正が発覚した場合、ビットコイン自体の価値が暴落すると考えられるため、保有している資産価値が激減してしまう。  
このような背景があり、ブロックチェーンは不正をするインセンティブを減らし、正規のマイニングをするインセンティブを与えることで正しく機能することができる。  
ちなみに、Proof of Workでは最も長いチェーンが正しいチェーンであるという決まりがある。  
これは最長のチェーンは多くのマシンによってたくさんの計算(仕事、Work)が投入されたチェーンだとみなすことができるから。

# ブロックヘッダを作る
Proof of Work はブロックヘッダをハッシュ化することで計算される。そこでブロックヘッダの生成についてここで確認する。
## ブロックヘッダの復習
ブロックヘッダは以下を含む80バイトのデータのこと。  

In [3]:
import pandas as pd
list = [
    ['Version', 'ビットフィールド', '4バイト'],
    ['prev block hash', '1つ前のブロックのハッシュ値', '32バイト'],
    ['Merkleroot', 'ハッシュ関数を使って取引をようやくしたハッシュ値', '32バイト'],
    ['Time', 'ブロックが生成された時間を示すタイムスタンプ', '4バイト'],
    ['Difficulty bits', 'マイニングの難易度', '4バイト'],
    ['Nonce', 'マイニングで条件を満たしたNonce(ナンス)値', '4バイト'],
]
df = pd.DataFrame(list, columns=['データ','説明', 'サイズ'])
df

,データ,説明,サイズ
0,Version,ビットフィールド,4バイト
1,prev block hash,1つ前のブロックのハッシュ値,32バイト
2,Merkleroot,ハッシュ関数を使って取引をようやくしたハッシュ値,32バイト
3,Time,ブロックが生成された時間を示すタイムスタンプ,4バイト
4,Difficulty bits,マイニングの難易度,4バイト
5,Nonce,マイニングで条件を満たしたNonce(ナンス)値,4バイト


# Nonceを変えてハッシュ計算
Nonceを除くブロックヘッダが生成できたら、Nonceを変えながらハッシュ計算を行う。　　
この時、一定の条件を満たすハッシュ値が見つかるまで繰り返す。  
## ハッシュ計算のイメージ
ハッシュ関数は入力値が少し変わるだけでも出力されるハッシュ値は大きく変化する。  
そのため、出力値から入力値を推測することは不可能だとされている。  
マイナーはハッシュ計算を行う際に、Nonceを少しずつ変えながら総当たりで計算する他ない。  
次にハッシュ関数において、入力値の変化に対してどのくらい出力値が変わるのかをプログラムで確認する。

In [4]:
import hashlib

input_text = "satoshi"

for nonce in range(20):
    input_data = input_text + str(nonce)
    hash = hashlib.sha256(input_data.encode('UTF-8')).hexdigest()
    print(input_data + " -> " + hash)

satoshi0 -> 4f9f790663d4a6cdb46c5636c7553f6239a2eca03194bb8e704c47718670faa0
satoshi1 -> 0952ab89f7cb3010a3048adcbd58b43c9e82c61622ac2d8893ff7c568f55c0bf
satoshi2 -> 43c1badd8834f74a03471ef4f51e2d9f69c4fb9bbb476aeb2bd6ac4e41432aa1
satoshi3 -> b8a25eafd4a22ba5d4510cfd5afe628a3497e42cd18bf8d35e736c260d967a27
satoshi4 -> be36eed769f4fa336843a5808bda6ef029d5cc9a917e14bb38f6d740cefc29ac
satoshi5 -> 1db1ebb2c528d0527b609b27c1d363f057e64d0c6e045dda339c860d726b23d8
satoshi6 -> 445cb342afee90ebbd82490006a54f70646e1be201961d849c033bbaf36dcba1
satoshi7 -> 8dabdf2893aa4693f36da21a5a32ddbf944f1a7e405c5c5ce30cdfa2458b99e4
satoshi8 -> ca35ad0c5f723110b34510b7396614f4d0b4a94081db918252009a53cf830da6
satoshi9 -> 659edacbf0973449d9e1b3fa5c47f15411acd7ffa52a8d334dd1c7fd546a4cd5
satoshi10 -> 7388c143e1d4a951fb684798f39c30136019624eb8a339e176c4fd6bae4f01ef
satoshi11 -> 7ad1b4eb050e4538e9fde9ee378ce7f979bd922d1e6ba9576c6ee5ac63648739
satoshi12 -> 18d541974dd684ae87f69f18debafed9bd44ab50b76d27b5175ec58052468

## Difficulty bits と Difficulty Target
Proof of Work において計算されるハッシュ値が満たすべき条件はどのように設定されているのでしょうか。  
その答えはブロックヘッダのDifficulty bitsにあります。 
この４バイトのフィールドには、ハッシュ値が満たすべき条件が整理されています。  
Difficulty bits は 0xle777777 のような形で記述されており、前半１バイトと後半３バイトに分解して解釈する。  
前半１バイトはexponent(指数部), 後半3バイトはcoefficient(係数部)やvalueと表現され、「右からexponentバイト目から、coefficentが始まる32バイトの値」と読み替えることができる。  
Proof of Work においてブロックヘッダは Double-SHA256でハッシュ化されるため、出力されるハッシュ値は32バイトとなる。  
32バイトのハッシュ値が,「右からexponentバイト目から、coefficientが始まる32バイトの値」よりも小さくなることが成功の条件。  
この時、「右からexponentバイト目から、coefficientが始まる32バイトの値」を Difficulty Target と言う。  
実際にbitsからTargetに変換する際は、Target = coefficient * 2⁸⁽ᵉˣᵖᵒⁿᵉⁿᵗ⁻³⁾ を用いる。

In [6]:
# この式を用いて 0x1e777777 を bits から Target に変換する。

# difficulty_bits = 0x1e777777
# exponent = 0x1e
# coefficient = 0x777777

target = 0x1e777777 * 2**(8*(0x1e - 0x03))
print(target)

# 10進数　-> 16進数
target_hex = hex(target)[2:].zfill(64)
print(target_hex)

53829940524435706462644443396939171196497807642659479737182017014036692992
001e777777000000000000000000000000000000000000000000000000000000


上のコードでは、出力の上段の数値は Difficultuy Target の10進数表示、下段の数値は１６進数表示。  
なお、１６進数表示は、０パディングして全部で３２バイトになるようにしている。  
.zfill() は引数で指定した桁数になるように右詰で０でパディングするメソッドです。  
また、hex(target)[2:]として最初の２文字を除外しているのは、１６進数表示を示す0xが含まれてしまうため。
## ブロックヘッダのハッシュ化
ブロックヘッダを生成し、nonceの値を少しずつ変えてハッシュ値を計算するプログラムの一例。

In [8]:
import hashlib

# ブロックのインデックスや１つ前のブロックのハッシュ値などの変数を定義
class Block():
    def __init__(self, data, prev_hash):
        self.index = 0
        self.nonce = 0
        self.prev_hash = prev_hash
        self.data = data

    # ブロックヘッダを作成後, sha256でハッシュ化してブロックヘッダのハッシュ値を求め、返り値としてblock_hashを設定
    def blockhash(self):
        blockheader = str(self.index) + str(self.prev_hash) + str(self.data) + str(self.nonce)
        block_hash = hashlib.sha256(blockheader.encode()).hexdigest()
        return block_hash

    # ブロックの情報を出力するようにしている
    def __str__(self):
        return "Block Hash: " + self.blockhash() + "\nPrevious Hash: " + self.prev_hash + "\nindex: " + str(self.index) + "\nData: " + str(self.data) + "\nNonce: " + str(self.nonce) + "\n--------------"

# 算出されるハッシュ値の処理を定義。
class Hashchain():
     
    # 変数として chain を定義し、配列を用意
    def __init__(self):
        # 最初のマイニングではprev_hashとして参照できるハッシュ値が存在しないため、全てが０の値を格納している。
        self.chain = ["0000000000000000000000000000000000000000000000000000000000000000"]

    # 算出されたハッシュ値を用意した chain の配列に格納していく処理を定義。
    def add(self, hash):
        self.chain.append(hash)

# Hashchainクラスをhashchainとしてインスタンス化
hashchain = Hashchain()
# マイニングの難易度としてtargetを計算
target = 0x777777 * 2**(8*(0x1e - 0x03)) # bits = 1e777777　の場合のターゲットの算出

# Blockクラスからblockをインスタンス化し、そのインスタンスのインデックス値を１つ増やす。
for i in range(30): # 全部で30個のブロックをマイニング
    block = Block("Block " + str(i+1), hashchain.chain[-1]) # ブロックの情報をデータとしてインスタンス化
    block.index = block.index + i + 1 # インデックスを設定
    # 4294967296を設定しているのは、Nonceが４バイトの１６進数で表現されるため、４バイトで表現できるNonceは１６の８乗になるから。なお、４バイトの１６進数で表現できる最大の数はffffff。
    for n in range(4294967296): # 2**32をナンスの上限としてナンスの計算を繰り返します。下のif文で条件に合わなかった場合、ナンス値を更新するのを繰り返す。
        block.nonce = block.nonce + n # ナンスの更新を行っています
        # int() は第一引数に文字列、第二引数に何進法とみなすかを指定する。
        if int(block.blockhash(), 16) < target: # ターゲットよりもハッシュ値が小さくなると条件クリアでブロックのデータが出力される
            print(block)
            hashchain.add(block.blockhash())
            break

"""
for s in range(len(hashchain.chain)):
    print(hashchain.chain[s])
"""

Block Hash: 0000145f739cf0536649fe385c9528d5c3786696c0b52172827bcb0c6a4fd766
Previous Hash: 0000000000000000000000000000000000000000000000000000000000000000
index: 1
Data: Block 1
Nonce: 2961844095
--------------
Block Hash: 0000176c37da63b6114c18008df3ff0975f9881b7d6f7923da61e0195b5387ae
Previous Hash: 0000145f739cf0536649fe385c9528d5c3786696c0b52172827bcb0c6a4fd766
index: 2
Data: Block 2
Nonce: 4239501321
--------------
Block Hash: 00004b4ccebb56f8babad59fc5f68c709b8e3b3f21fcab250ff9437ec8cc2714
Previous Hash: 0000176c37da63b6114c18008df3ff0975f9881b7d6f7923da61e0195b5387ae
index: 3
Data: Block 3
Nonce: 560605870
--------------
Block Hash: 000033c56771fbd4974611dfbdf61cc3683315bc20df488bac30203f34e56daf
Previous Hash: 00004b4ccebb56f8babad59fc5f68c709b8e3b3f21fcab250ff9437ec8cc2714
index: 4
Data: Block 4
Nonce: 4668243625
--------------
Block Hash: 0000248287c0af0f708d898418f335a6b1e60b8a26c52fe9e6effbba4bf622da
Previous Hash: 000033c56771fbd4974611dfbdf61cc3683315bc20df488bac30203f3

## 条件に合うハッシュ値が見つからない場合
ところで、Nonceはブロックヘッダのうち、４バイトを占めている。  
１６進数で表現されるため、４バイトで表現できるNonceは１６の８乗の約４３億通りとなる。  
条件に合うハッシュ値がこのNonceで発見できるかどうかわかりませんし、見つからない場合もあり、その場合はいくつかの方法で調整して計算を行う。  
簡単な例として、ブロックヘッダのタイムスタンプを調整するケースがある。  
マイニングは世界中のマシンを用いて行われるため、それぞれのマシン内の時計が互いにずれている可能性は高いと言える。  
実際のブロックチェーンで利用されているタイムスタンプは正しくなく、ブロックの前後関係とタイムスタンプの前後関係が逆転している場合すらある。  
そのため、タイムスタンプを少しずつ調整することでハッシュの計算値を調整することもある。  
また、Extra Nonce を利用するケースもある。  
Extra Nonce とは、ブロックヘッダ以外に含まれるナンスのことで、具体的にはコインベース取引のinput領域にあるcoinbase script の８バイトを利用する。  
コインベース取引はトランザクションデータであるため、他のデータとまとめて要約されてマークルルートに集約される。  
当然コインベース取引のデータを変更すれば、マークルルートの値も変動し、それによりブロックヘッダのハッシュ値も変わる。  
つまり、コインベース取引のデータもブロックヘッダにあるNonceと同じように調整対象と見なすことができる。  